# Introduction to Docker

**Learning Objectives**
  1. Build and run Docker containers
  2. Pull Docker images from Docker Hub and Google Container Registry
  3. Push Docker images to Google Container Registry

## Overview

Docker is an open platform for developing, shipping, and running applications. With Docker, you can separate your applications from your infrastructure and treat your infrastructure like a managed application. Docker helps you ship code faster, test faster, deploy faster, and shorten the cycle between writing code and running code.

Docker does this by combining kernel containerization features with workflows and tooling that helps you manage and deploy your applications.

Docker containers can be directly used in Kubernetes, which allows them to be run in the Kubernetes Engine with ease. After learning the essentials of Docker, you will have the skillset to start developing Kubernetes and containerized applications.

## Basic Docker commands

See what docker images you have. 

In [1]:
!docker images

REPOSITORY                     TAG                 IMAGE ID            CREATED             SIZE
gcr.io/inverting-proxy/agent   <none>              f19154b89377        9 months ago        1.07GB


If this is the first time working with docker you won't have any repositories listed. Let's use `docker run` to pull a docker image called `hello-world` from the public registry. The docker daemon will search for the `hello-world` image, if it doesn't find the image locally, it pulls the image from a public registry called Docker Hub, creates a container from that image, and runs the container for you.

In [2]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

bdcc26d7: Pull complete 529kB/2.529kBBDigest: sha256:8c5aeeb6a5f3ba4883347d3747a7249f491766ca1caa47e5da5dfcf6b9b717c0
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For 

Now when we look at our docker images we should see `hello-world` there as well.

In [3]:
!docker images

REPOSITORY                     TAG                 IMAGE ID            CREATED             SIZE
gcr.io/inverting-proxy/agent   <none>              f19154b89377        9 months ago        1.07GB
hello-world                    latest              bf756fb1ae65        10 months ago       13.3kB


This is the image pulled from the Docker Hub public registry. The Image ID is in `SHA256` hash format—this field specifies the Docker image that's been provisioned. When the docker daemon can't find an image locally, it will by default search the public registry for the image. Let's run the container again:

Now, if we want to run `docker run hello-world` again, it won't have to download from the container registry.

To see all docker containers running, use `docker ps`.

In [4]:
!docker ps

CONTAINER ID        IMAGE                          COMMAND                  CREATED             STATUS              PORTS               NAMES
4bd7c1e217b4        gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   6 minutes ago       Up 6 minutes                            proxy-agent


There are no running containers. The `hello-world` containers you ran previously already exited. In order to see all containers, including ones that have finished executing, run docker `ps -a`:

In [5]:
!docker ps -a

CONTAINER ID        IMAGE                          COMMAND                  CREATED             STATUS                     PORTS               NAMES
420aa95c9828        hello-world                    "/hello"                 10 seconds ago      Exited (0) 9 seconds ago                       flamboyant_engelbart
4bd7c1e217b4        gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   6 minutes ago       Up 6 minutes                                   proxy-agent


This shows you the Container ID, a UUID generated by Docker to identify the container, and more metadata about the run. The container Names are also randomly generated but can be specified with docker run --name [container-name] hello-world.

## Build a Docker container

Let's build a Docker image that's based on a simple node application. Execute the following command to create and switch into a folder named test.



Open a new text file and write the following. Save the file as `Dockerfile`

```bash
# Use an official Node runtime as the parent image
FROM node:6

# Set the working directory in the container to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
ADD . /app

# Make the container's port 80 available to the outside world
EXPOSE 80

# Run app.js using node when the container launches
CMD ["node", "app.js"]
EOF
```

This file instructs the Docker daemon on how to build your image.

The initial line specifies the base parent image, which in this case is the official Docker image for node version 6.
In the second, we set the working (current) directory of the container.
In the third, we add the current directory's contents (indicated by the "." ) into the container.
Then we expose the container's port so it can accept connections on that port and finally run the node command to start the application.

Check out the other [Docker command references](https://docs.docker.com/engine/reference/builder/#known-issues-run) to understand what each line does.

We're going to use this Docker container to run a simple node.js app. Have a look at `app.js`. This is a simple HTTP server that listens on port 80 and returns "Hello World."


Now let's build the image. Note again the "`.`", which means current directory so you need to run this command from within the directory that has the Dockerfile.

The `-t` is to name and tag an image with the `name:tag` syntax. The name of the image is `node-app` and the tag is `0.1`. The tag is highly recommended when building Docker images. If you don't specify a tag, the tag will default to latest and it becomes more difficult to distinguish newer images from older ones. Also notice how each line in the Dockerfile above results in intermediate container layers as the image is built.

In [6]:
!docker build -t node-app:0.1 -f dockerfiles/intro.docker .

Sending build context to Docker daemon  61.95kB
Step 1/5 : FROM node:6
6: Pulling from library/node

55d5a1d1: Pulling fs layer 
80d00ae9: Pulling fs layer 
b3117dca: Pulling fs layer 
a19181b2: Pulling fs layer 
7b2a5bcc: Pulling fs layer 
12c70287: Pulling fs layer 
5386a42d: Pulling fs layer 
Digest: sha256:e133e66ec3bfc98da0440e552f452e5cdf6413319d27a2db3b01ac4b319759b3[5AExtracting  73.53MB/215.1MB
Status: Downloaded newer image for node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Running in 84a331f75800
Removing intermediate container 84a331f75800
 ---> 4e1563a2c1ec
Step 3/5 : ADD . /app
 ---> 83146e857575
Step 4/5 : EXPOSE 80
 ---> Running in 580e11c557ea
Removing intermediate container 580e11c557ea
 ---> 91393ad74d72
Step 5/5 : CMD ["node", "app.js"]
 ---> Running in cafc8bc5bbef
Removing intermediate container cafc8bc5bbef
 ---> 95e54f75598e
Successfully built 95e54f75598e
Successfully tagged node-app:0.1


Let's check that the image has been created correctly. 

In [7]:
!docker images

REPOSITORY                     TAG                 IMAGE ID            CREATED             SIZE
node-app                       0.1                 95e54f75598e        3 seconds ago       884MB
gcr.io/inverting-proxy/agent   <none>              f19154b89377        9 months ago        1.07GB
hello-world                    latest              bf756fb1ae65        10 months ago       13.3kB
node                           6                   ab290b853066        18 months ago       884MB


You should see a `node-app` repository that was created only seconds ago. 

Notice `node` is the base image and `node-app` is the image you built. You can't remove `node` without removing `node-app` first. The size of the image is relatively small compared to VMs. Other versions of the node image such as `node:slim` and `node:alpine` can give you even smaller images for easier portability. The topic of slimming down container sizes is further explored in Advanced Topics. You can view all versions in the official repository here.

Note, you can remove an image from your docker images using `docker rmi [repository]:[tag]`.

## Run a Docker container

Now we'll run the container based on the image you built above using the `docker run` command. The `--name` flag allows you to name the container if you like. And `-p` instructs Docker to map the host's port 4000 to the container's port 80. This allows you to reach the server at http://localhost:4000. Without port mapping, you would not be able to reach the container at localhost.

In [8]:
!docker run -p 4000:80 --name my-app node-app:0.1

Server running at http://0.0.0.0:80/
^C
Caught interrupt signal and will exit


To test out the server, open a terminal window and type the following command:

```bash
curl http://localhost:4000
```

You should see the server respond with `Hello World`

The container will run as long as the initial terminal is running. If you want to stop the container, run the following command in the terminal to stop and remove the container:

```bash
docker stop my-app && docker rm my-app
```
After a few moments the container will stop.

#### Running the container in the background
If you want to the container to run in the background (not tied to the terminal's session), you need to specify the `-d` flag.
Now run the following command to start the container in the background

In [16]:
!docker run -p 4000:80 --name my-app -d node-app:0.1

b01226c345cf8540434eb13d0521a3d8c3e1a28253742703f566cb63459ca632


Your container is now running in the background. You can check the status of your running container using `docker ps`

In [17]:
!docker ps

CONTAINER ID        IMAGE                          COMMAND                  CREATED             STATUS              PORTS                  NAMES
b01226c345cf        node-app:0.1                   "node app.js"            16 seconds ago      Up 15 seconds       0.0.0.0:4000->80/tcp   my-app
4bd7c1e217b4        gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   10 minutes ago      Up 10 minutes                              proxy-agent


Notice the container is running in the output of docker ps. You can look at the logs by executing `docker logs [container_id]`. 

In [18]:
# Note, your container id will be different
!docker logs b01226c345cf

Server running at http://0.0.0.0:80/


You should see 
```bash
Server running at http://0.0.0.0:80/
```
If you want to follow the log's output as the container is running, use the `-f` option.

## Modify & Publish

Let's modify the application and push it to your Google Cloud Repository (gcr). After that you'll remove all local containers and images to simulate a fresh environment, and then pull and run your containers from gcr. This will demonstrate the portability of Docker containers.

### Edit `app.js`
Open the file `app.js` with the text editor and replace "Hello World" with another string. Then build this new image. 

In [19]:
!docker build -t node-app:0.2 .

Sending build context to Docker daemon  67.07kB
Step 1/5 : FROM node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> 4e1563a2c1ec
Step 3/5 : ADD . /app
 ---> 1fd82f4a2a4c
Step 4/5 : EXPOSE 80
 ---> Running in a5b0078d0dc6
Removing intermediate container a5b0078d0dc6
 ---> b32b4e08312b
Step 5/5 : CMD ["node", "app.js"]
 ---> Running in d271d1ef62a2
Removing intermediate container d271d1ef62a2
 ---> 868790a6e8cc
Successfully built 868790a6e8cc
Successfully tagged node-app:0.2


Notice in `Step 2` of the output we are using an existing cache layer. From `Step 3` and on, the layers are modified because we made a change in `app.js`.

Run another container with the new image version. Notice how we map the host's port 8000 instead of 80. We can't use host port 4000 because it's already in use. 

In [20]:
!docker run -p 8000:80 --name my-app-2 -d node-app:0.2

a694dc8bc9476bcafadcd9212eb21553a150bdf21f28bf3d08a5c0aea540c45f


You can check that both container are running using `docker ps`.

In [21]:
!docker ps

CONTAINER ID        IMAGE                          COMMAND                  CREATED              STATUS              PORTS                  NAMES
a694dc8bc947        node-app:0.2                   "node app.js"            2 seconds ago        Up 1 second         0.0.0.0:8000->80/tcp   my-app-2
b01226c345cf        node-app:0.1                   "node app.js"            About a minute ago   Up About a minute   0.0.0.0:4000->80/tcp   my-app
4bd7c1e217b4        gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   11 minutes ago       Up 11 minutes                              proxy-agent


And let's test boht containers using `curl` as before:

In [22]:
!curl http://localhost:8000

Hello ASL


In [23]:
!curl http://localhost:4000

Hello ASL


Recall, to stop a container running, you can execute the following command either in a terminal or in a cell in this notebook:

```bash
docker stop [name] && docker rm [name]
```

### Publish to gcr

Now you're going to push your image to the Google Container Registry (gcr). To push images to your private registry hosted by gcr, you need to tag the images with a registry name. The format is `[hostname]/[project-id]/[image]:[tag]`.

For gcr:

  * `[hostname]`= gcr.io
  * `[project-id]`= your project's ID
  * `[image]`= your image name
  * `[tag]`= any string tag of your choice. If unspecified, it defaults to "latest".

In [2]:
import os

PROJECT_ID = 'munn-sandbox'

os.environ["PROJECT_ID"] = PROJECT_ID

Let's tag `node-app:0.2`.

In [25]:
!docker images

REPOSITORY                     TAG                 IMAGE ID            CREATED             SIZE
node-app                       0.2                 868790a6e8cc        2 minutes ago       884MB
node-app                       0.1                 95e54f75598e        6 minutes ago       884MB
gcr.io/inverting-proxy/agent   <none>              f19154b89377        9 months ago        1.07GB
hello-world                    latest              bf756fb1ae65        10 months ago       13.3kB
node                           6                   ab290b853066        18 months ago       884MB


In [26]:
%%bash
docker tag node-app:0.2 gcr.io/${PROJECT_ID}/node-app:0.2

Now when we list our docker images we should see this newly tagged repository.

In [27]:
!docker images

REPOSITORY                     TAG                 IMAGE ID            CREATED             SIZE
node-app                       0.2                 868790a6e8cc        2 minutes ago       884MB
gcr.io/munn-sandbox/node-app   0.2                 868790a6e8cc        2 minutes ago       884MB
node-app                       0.1                 95e54f75598e        6 minutes ago       884MB
gcr.io/inverting-proxy/agent   <none>              f19154b89377        9 months ago        1.07GB
hello-world                    latest              bf756fb1ae65        10 months ago       13.3kB
node                           6                   ab290b853066        18 months ago       884MB


Next, let's push this image to gcr.

In [28]:
%%bash
docker push gcr.io/${PROJECT_ID}/node-app:0.2

The push refers to repository [gcr.io/munn-sandbox/node-app]
b0148ef49e6e: Preparing
f707bb0943ea: Preparing
f39151891503: Preparing
f1965d3c206f: Preparing
a27518e43e49: Preparing
910d7fd9e23e: Preparing
4230ff7f2288: Preparing
2c719774c1e1: Preparing
ec62f19bb3aa: Preparing
f94641f1fe1f: Preparing
4230ff7f2288: Waiting
2c719774c1e1: Waiting
ec62f19bb3aa: Waiting
910d7fd9e23e: Waiting
f94641f1fe1f: Waiting
f39151891503: Mounted from asl-ml-immersion/node-app
a27518e43e49: Mounted from asl-ml-immersion/node-app
f1965d3c206f: Mounted from asl-ml-immersion/node-app
910d7fd9e23e: Mounted from asl-ml-immersion/node-app
2c719774c1e1: Mounted from asl-ml-immersion/node-app
4230ff7f2288: Mounted from asl-ml-immersion/node-app
b0148ef49e6e: Pushed
f707bb0943ea: Pushed
ec62f19bb3aa: Mounted from asl-ml-immersion/node-app
f94641f1fe1f: Mounted from asl-ml-immersion/node-app
0.2: digest: sha256:d766b881e3a5ad52a86bada1c1c670237f0233c37c382601bf67814bc64cda15 size: 2423


Check that the image exists in gcr by visiting the image registry Cloud Console. You can navigate via the console to `Navigation menu > Container Registry` or visit the url from the cell below:

In [29]:
%%bash
echo "http://gcr.io/${PROJECT_ID}/node-app"

http://gcr.io/munn-sandbox/node-app


### Test the published gcr image

Let's test this image. You could start a new VM, ssh into that VM, and install gcloud. For simplicity, we'll just remove all containers and images to simulate a fresh environment.

First, stop and remove all containers. From the terminal run

```bash
docker stop $(docker ps -q)
docker rm $(docker ps -aq)
```

Now remove the docker images you've created above. 

In [ ]:
%%bash
docker rmi node-app:0.2 gcr.io/${PROJECT_ID}/node-app node-app:0.1
docker rmi node:6

Use `docker rmi` to remove any remaining docker images. Confirm with `docker images`.

In [7]:
!docker images

REPOSITORY                     TAG                 IMAGE ID            CREATED             SIZE
gcr.io/inverting-proxy/agent   <none>              f19154b89377        9 months ago        1.07GB


At this point you should have a pseudo-fresh environment. Now, pull the image and run it.

In [8]:
%%bash
docker pull gcr.io/${PROJECT_ID}/node-app:0.2
docker run -p 4000:80 -d gcr.io/${PROJECT_ID}/node-app:0.2

0.2: Pulling from munn-sandbox/node-app
c5e155d5a1d1: Pulling fs layer
221d80d00ae9: Pulling fs layer
4250b3117dca: Pulling fs layer
3b7ca19181b2: Pulling fs layer
425d7b2a5bcc: Pulling fs layer
69df12c70287: Pulling fs layer
ea2f5386a42d: Pulling fs layer
d421d2b3c5eb: Pulling fs layer
cbe76f3886f3: Pulling fs layer
6d09f8080287: Pulling fs layer
69df12c70287: Waiting
3b7ca19181b2: Waiting
ea2f5386a42d: Waiting
d421d2b3c5eb: Waiting
425d7b2a5bcc: Waiting
6d09f8080287: Waiting
cbe76f3886f3: Waiting
4250b3117dca: Verifying Checksum
4250b3117dca: Download complete
221d80d00ae9: Verifying Checksum
221d80d00ae9: Download complete
c5e155d5a1d1: Download complete
69df12c70287: Verifying Checksum
69df12c70287: Download complete
ea2f5386a42d: Verifying Checksum
ea2f5386a42d: Download complete
3b7ca19181b2: Verifying Checksum
3b7ca19181b2: Download complete
cbe76f3886f3: Verifying Checksum
cbe76f3886f3: Download complete
d421d2b3c5eb: Verifying Checksum
d421d2b3c5eb: Download complete
6d09f8080

You can check that it's running as expected using before:

In [9]:
!curl http://localhost:4000

Hello ASL


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.